In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [4]:
# Load the dataset
file_path = '/content/Infrared.csv'  # Ganti dengan path file dataset Anda
data = pd.read_csv(file_path)

data.head()

,Gender,Age,Ethnicity,T_atm,Humidity,Distance,T_offset1,Max1R13_1,Max1L13_1,aveAllR13_1,...,T_FHRC1,T_FHLC1,T_FHBC1,T_FHTC1,T_FH_Max1,T_FHC_Max1,T_Max1,T_OR1,T_OR_Max1,aveOralM
0,Male,41-50,White,24.0,28.0,0.8,0.7025,35.0300,35.3775,34.4000,...,33.4775,33.3725,33.4925,33.0025,34.5300,34.0075,35.6925,35.6350,35.6525,36.59
1,Female,31-40,Black or African-American,24.0,26.0,0.8,0.7800,34.5500,34.5200,33.9300,...,34.0550,33.6775,33.9700,34.0025,34.6825,34.6600,35.1750,35.0925,35.1075,37.19
2,Female,21-30,White,24.0,26.0,0.8,0.8625,35.6525,35.5175,34.2775,...,34.8275,34.6475,34.8200,34.6700,35.3450,35.2225,35.9125,35.8600,35.8850,37.34
3,Female,21-30,Black or African-American,24.0,27.0,0.8,0.9300,35.2225,35.6125,34.3850,...,34.4225,34.6550,34.3025,34.9175,35.6025,35.3150,35.7200,34.9650,34.9825,37.09
4,Male,18-20,White,24.0,27.0,0.8,0.8950,35.5450,35.6650,34.9100,...,35.1600,34.3975,34.6700,33.8275,35.4175,35.3725,35.8950,35.5875,35.6175,37.04


In [8]:
# Separating features and target variable
X = data.drop(columns=['aveOralM'])
y = data['aveOralM']

# Identifying categorical and numerical features
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=[np.number]).columns

In [11]:
# Preprocessing the data (Handling missing values, encoding categorical features, and scaling)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Creating a pipeline for K-NN model
pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=5))
])

# Creating a pipeline for Decision Tree model
pipeline_dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('model', DecisionTreeRegressor(random_state=42))
])

# Training the models
pipeline_knn.fit(X_train, y_train)
pipeline_dt.fit(X_train, y_train)

# Making predictions
knn_predictions = pipeline_knn.predict(X_test)
dt_predictions = pipeline_dt.predict(X_test)

In [13]:
# Evaluating the models
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, r2

# Evaluating K-NN and Decision Tree
knn_mse, knn_rmse, knn_r2 = evaluate_model(y_test, knn_predictions)
dt_mse, dt_rmse, dt_r2 = evaluate_model(y_test, dt_predictions)

# Print the results
print("K-NN Model Evaluation:")
print(f"MSE: {knn_mse}")
print(f"RMSE: {knn_rmse}")
print(f"R²: {knn_r2}")

print("\nDecision Tree Model Evaluation:")
print(f"MSE: {dt_mse}")
print(f"RMSE: {dt_rmse}")
print(f"R²: {dt_r2}")

K-NN Model Evaluation:
MSE: 0.07271862745098044
RMSE: 0.26966391573768345
R²: 0.6546616806475278

Decision Tree Model Evaluation:
MSE: 0.12643382352941193
RMSE: 0.35557534156548587
R²: 0.39957001861198527


### 1. **Mean Squared Error (MSE)**

MSE adalah ukuran yang digunakan untuk menghitung rata-rata dari kuadrat perbedaan antara nilai aktual dan prediksi yang dihasilkan oleh model. Persamaan MSE adalah sebagai berikut:

\[
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
\]

- \( n \) adalah jumlah sampel (data points).
- \( y_i \) adalah nilai aktual dari target (observasi yang sebenarnya).
- \( \hat{y}_i \) adalah nilai prediksi yang dihasilkan oleh model.
- \( (y_i - \hat{y}_i)^2 \) adalah kuadrat dari selisih antara nilai aktual dan prediksi.

**Penjelasan**:
- MSE memberikan gambaran tentang seberapa besar kesalahan prediksi model. Semakin kecil nilai MSE, semakin baik performa model.
- MSE cenderung sangat sensitif terhadap outlier, karena perbedaan yang besar akan dikuadratkan.

---

### 2. **Root Mean Squared Error (RMSE)**

RMSE adalah akar kuadrat dari MSE, dan memberikan gambaran yang lebih langsung dalam satuan yang sama dengan target variabel (seperti nilai asli dari data). Persamaannya adalah:

\[
RMSE = \sqrt{MSE}
\]

**Penjelasan**:
- RMSE adalah cara untuk mengukur kesalahan prediksi dalam unit yang sama dengan target variabel.
- RMSE lebih mudah untuk diinterpretasikan daripada MSE karena tidak dalam kuadrat unit, dan tetap memberikan informasi yang sama tentang seberapa besar kesalahan prediksi.

---

### 3. **R-Squared (R²)**

R-squared (atau koefisien determinasi) mengukur seberapa baik model menjelaskan variasi dalam data. Ini menunjukkan seberapa besar proporsi varians dalam data yang dapat dijelaskan oleh model yang dibangun. Persamaannya adalah sebagai berikut:

\[
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
\]

- \( y_i \) adalah nilai aktual dari target.
- \( \hat{y}_i \) adalah nilai prediksi dari model.
- \( \bar{y} \) adalah rata-rata dari nilai aktual \( y_i \).
- \( \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 \) adalah jumlah kuadrat kesalahan (residual sum of squares, RSS).
- \( \sum_{i=1}^{n} (y_i - \bar{y})^2 \) adalah jumlah kuadrat total variasi (total sum of squares, TSS).

**Penjelasan**:
- R² memberikan informasi tentang seberapa baik model memprediksi data yang ada. Nilai R² berkisar antara 0 dan 1:
  - Jika R² = 1, model menjelaskan semua variasi dalam data.
  - Jika R² = 0, model tidak menjelaskan variasi lebih baik daripada rata-rata dari target.
- R² yang lebih tinggi menunjukkan bahwa model lebih baik dalam memprediksi nilai target.

---

### Kesimpulan:
- **MSE** dan **RMSE** mengukur kesalahan model dalam memprediksi target, dengan RMSE memberikan gambaran yang lebih mudah dipahami dalam satuan yang sama dengan target.
- **R²** memberikan gambaran tentang seberapa baik model dapat menjelaskan variasi dalam data, dengan nilai yang lebih tinggi menunjukkan performa model yang lebih baik.
